In [ ]:
#Date: Dec 11, 2024
#Author: Sonal Allana
#Purpose: To generate IG and SG explanations from the models

In [1]:
import numpy as np
from numpy import loadtxt
from numpy import savetxt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time

tf.compat.v1.disable_eager_execution() #needed for INNvestigate to work

import innvestigate
import innvestigate.utils 
import os
import miattack

2024-10-10 22:24:15.623876: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Options (1) baseline_nn (2) dp_nn (3) syn_nn
base_folder = "dp_nn"

if base_folder == "dp_nn":
    #Options (1) nm4000 (2) nm500 (3) nm66 (4) nm15
    nm_folder = "nm4000"   #set the correct noise multiplier 
elif base_folder == "syn_nn":
    #Options (1) ctgan (2) gausscopula (3) tvae 
    syndataType = "gausscopula"


In [3]:
#Options (1) adult (2) credit (3) compas (4) hospital
dataset_name = "hospital"


In [4]:
#Uncomment for gradient
#expl = "gradient"
#expl_folder = "grad"
#end gradient

#Uncomment for integrated gradient
#expl = "integrated_gradients"
#expl_folder = "ig"
#end integrated gradient

#Uncomment for smoothgrad
expl = "smoothgrad"
expl_folder = "sg"
#end smoothgrad

In [5]:
for i in range(1,6):
    #Load model and train-test files
    if base_folder == "dp_nn": #account for noise multiplier folder
        basepath = "../models/{0}/{1}/{2}/iter{3}/".format(base_folder,dataset_name,nm_folder,i)
        fmodel = "model_dp_iter{0}.keras".format(i)
    elif base_folder == "syn_nn": #account for syn type folder
        basepath = "../models/{0}/{1}/{2}/iter{3}/".format(base_folder,dataset_name,syndataType,i)
        fmodel = "model_wodp_iter{0}.keras".format(i)
    else:
        basepath = "../models/{0}/{1}/iter{2}/".format(base_folder,dataset_name, i)
        fmodel = "model_wodp_iter{0}.keras".format(i)
    model = tf.keras.models.load_model(basepath + fmodel,compile=False)
    model.summary()
    X_train = loadtxt(basepath + 'X_train.csv',delimiter=',')
    X_test = loadtxt(basepath + 'X_test.csv',delimiter=',')
    Y_train = loadtxt(basepath + 'Y_train.csv',delimiter=',')
    Y_test = loadtxt(basepath + 'Y_test.csv',delimiter=',')
    Z_train = loadtxt(basepath + 'Z_train.csv',delimiter=',')
    Z_test = loadtxt(basepath + 'Z_test.csv',delimiter=',')
    
    #50% of the test set is used for training and remaining 50% for testing the attack model
    #For synthetic data, X_test and Y_test is original non-synthetic test data 
    #Synthetic models are trained on original data and tested on synthetic data - this is taken in account in SynNN.ipynb notebook
    (X_adv_train, X_adv_test, Y_adv_train, Y_adv_test, Z_adv_train, Z_adv_test)  = train_test_split(X_test, Y_test, Z_test, test_size=0.5, random_state=1337)

    input_train = np.array(X_adv_train)
    input_test = np.array(X_adv_test)
    analyzer = innvestigate.create_analyzer(expl, model)
    
    attributions_train = analyzer.analyze(input_train)
    print(np.shape(attributions_train))
    
    attributions_test = analyzer.analyze(input_test)
    print(np.shape(attributions_test))
    
    if not os.path.exists(basepath + expl_folder):
        os.mkdir(basepath + expl_folder)
    
    savetxt(basepath + expl_folder + '/attributions_train.csv',attributions_train,delimiter=',')
    savetxt(basepath + expl_folder + '/attributions_test.csv',attributions_test,delimiter=',')
    savetxt(basepath + expl_folder + '/X_adv_train.csv',X_adv_train,delimiter=',')
    savetxt(basepath + expl_folder + '/X_adv_test.csv',X_adv_test,delimiter=',')
    savetxt(basepath + expl_folder + '/Y_adv_train.csv',Y_adv_train,delimiter=',')
    savetxt(basepath + expl_folder + '/Y_adv_test.csv',Y_adv_test,delimiter=',')
    savetxt(basepath + expl_folder + '/Z_adv_train.csv',Z_adv_train,delimiter=',')
    savetxt(basepath + expl_folder + '/Z_adv_test.csv',Z_adv_test,delimiter=',')
    
    print("Iteration ",i,":")
    #Attack the first sensitive attribute
    print("Model inversion on sensitive attribute 1: ")
    Z_adv_train_s1 = Z_adv_train[:,0]
    Z_adv_test_s1 = Z_adv_test[:,0]
    modinv_obj1 = miattack.miattack_explanations(attributions_train,attributions_test,Z_adv_train_s1,Z_adv_test_s1)
    modinv_obj1.printMetrics()
    
    #Attack the second sensitive attribute
    print("\nModel inversion on sensitive attribute 2: ")
    Z_adv_train_s2 = Z_adv_train[:,1]
    Z_adv_test_s2 = Z_adv_test[:,1]
    modinv_obj2 = miattack.miattack_explanations(attributions_train,attributions_test,Z_adv_train_s2,Z_adv_test_s2)
    modinv_obj2.printMetrics()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 120)]             0         
                                                                 
 dense1 (Dense)              (None, 1024)              123904    
                                                                 
 dense2 (Dense)              (None, 512)               524800    
                                                                 
 dense3 (Dense)              (None, 256)               131328    
                                                                 
 dense4 (Dense)              (None, 100)               25700     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 805833 (3.07 MB)
Trainable params: 805833 (3.07

2024-10-10 22:24:38.846559: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2024-10-10 22:24:38.866412: W tensorflow/c/c_api.cc:305] Operation '{name:'dense4/bias/Assign' id:94 op device:{requested: '', assigned: ''} def:{{{node dense4/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense4/bias, dense4/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-10-10 22:24:38.919908: W tensorflow/c/c_api.cc:305] Operation '{name:'dense2/MatMul/ReadVariableOp' id:48 op device:{requested: '', assigned: ''} def:{{{node dense2/MatMul/ReadVariableOp}} = ReadVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT](dense2/kernel)}}' was changed by setting attribute after it was run by a s

(4925, 120)
(4925, 120)
Iteration  1 :
Model inversion on sensitive attribute 1: 
Precision: 0.8908
Recall: 0.8876
F1-Score: 0.8892
FPR: 0.0923
Attacker's advantage: 0.7953
Attack success: 89.85%

Model inversion on sensitive attribute 2: 
Precision: 0.9523
Recall: 0.9485
F1-Score: 0.9504
FPR: 0.1544
Attacker's advantage: 0.7940
Attack success: 92.43%
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 120)]             0         
                                                                 
 dense1 (Dense)              (None, 1024)              123904    
                                                                 
 dense2 (Dense)              (None, 512)               524800    
                                                                 
 dense3 (Dense)              (None, 256)               131328    
                                   

2024-10-10 22:25:59.360498: W tensorflow/c/c_api.cc:305] Operation '{name:'dense3_1/kernel/Assign' id:330 op device:{requested: '', assigned: ''} def:{{{node dense3_1/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense3_1/kernel, dense3_1/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-10-10 22:25:59.440001: W tensorflow/c/c_api.cc:305] Operation '{name:'dense2_1/MatMul/ReadVariableOp' id:313 op device:{requested: '', assigned: ''} def:{{{node dense2_1/MatMul/ReadVariableOp}} = ReadVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT](dense2_1/kernel)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either

(4925, 120)
(4925, 120)
Iteration  2 :
Model inversion on sensitive attribute 1: 
Precision: 0.8753
Recall: 0.8730
F1-Score: 0.8742
FPR: 0.1054
Attacker's advantage: 0.7676
Attack success: 88.47%

Model inversion on sensitive attribute 2: 
Precision: 0.9432
Recall: 0.9649
F1-Score: 0.9539
FPR: 0.1890
Attacker's advantage: 0.7760
Attack success: 92.87%
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 120)]             0         
                                                                 
 dense1 (Dense)              (None, 1024)              123904    
                                                                 
 dense2 (Dense)              (None, 512)               524800    
                                                                 
 dense3 (Dense)              (None, 256)               131328    
                                   

...

(4925, 120)
(4925, 120)
Iteration  3 :
Model inversion on sensitive attribute 1: 
Precision: 0.8478
Recall: 0.8748
F1-Score: 0.8611
FPR: 0.1332
Attacker's advantage: 0.7416
Attack success: 87.05%

Model inversion on sensitive attribute 2: 
Precision: 0.9462
Recall: 0.9623
F1-Score: 0.9542
FPR: 0.1777
Attacker's advantage: 0.7846
Attack success: 92.93%
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 120)]             0         
                                                                 
 dense1 (Dense)              (None, 1024)              123904    
                                                                 
 dense2 (Dense)              (None, 512)               524800    
                                                                 
 dense3 (Dense)              (None, 256)               131328    
                                   

2024-10-10 22:27:58.830897: W tensorflow/c/c_api.cc:305] Operation '{name:'dense4_3/bias/Assign' id:889 op device:{requested: '', assigned: ''} def:{{{node dense4_3/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense4_3/bias, dense4_3/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-10-10 22:27:58.978444: W tensorflow/c/c_api.cc:305] Operation '{name:'dense2_3/MatMul/ReadVariableOp' id:843 op device:{requested: '', assigned: ''} def:{{{node dense2_3/MatMul/ReadVariableOp}} = ReadVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT](dense2_3/kernel)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after r

(4925, 120)
(4925, 120)
Iteration  4 :
Model inversion on sensitive attribute 1: 
Precision: 0.8854
Recall: 0.8650
F1-Score: 0.8751
FPR: 0.0949
Attacker's advantage: 0.7701
Attack success: 88.67%

Model inversion on sensitive attribute 2: 
Precision: 0.9450
Recall: 0.9498
F1-Score: 0.9474
FPR: 0.1795
Attacker's advantage: 0.7703
Attack success: 91.94%
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 120)]             0         
                                                                 
 dense1 (Dense)              (None, 1024)              123904    
                                                                 
 dense2 (Dense)              (None, 512)               524800    
                                                                 
 dense3 (Dense)              (None, 256)               131328    
                                   

2024-10-10 22:29:26.611186: W tensorflow/c/c_api.cc:305] Operation '{name:'dense2_4/bias/Assign' id:1106 op device:{requested: '', assigned: ''} def:{{{node dense2_4/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense2_4/bias, dense2_4/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-10-10 22:29:26.801028: W tensorflow/c/c_api.cc:305] Operation '{name:'dense2_4/MatMul/ReadVariableOp' id:1108 op device:{requested: '', assigned: ''} def:{{{node dense2_4/MatMul/ReadVariableOp}} = ReadVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT](dense2_4/kernel)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after

(4925, 120)
(4925, 120)
Iteration  5 :
Model inversion on sensitive attribute 1: 
Precision: 0.8874
Recall: 0.8929
F1-Score: 0.8902
FPR: 0.0961
Attacker's advantage: 0.7969
Attack success: 89.89%

Model inversion on sensitive attribute 2: 
Precision: 0.9494
Recall: 0.9660
F1-Score: 0.9576
FPR: 0.1674
Attacker's advantage: 0.7986
Attack success: 93.46%
